## This script read alignment obtained by Gentle

Issue: syllables in the paradigm (spreadsheet) are broken into phonemes by Gentle, how to program to combine them?

In [1]:
import os
from scipy.io import wavfile
from dSPEECH.config import *
import json

2024-08-09 18:04:35 - Start Program

pre_all: Running from CMD.
common_dl.py: Using CUDA.


In [2]:
sid=2
folder=meta_dir+'SEEG'+str(sid)+'/forced_alignment/' #alignment/'

### checking

In [4]:
clip=0
json_file=folder+'/json/'+str(clip)+'.json'
sentences_file=meta_dir+'sentences/sentences.npy'

In [19]:
with open(json_file) as f:
    align_dict=json.load(f)
sentences=np.load(sentences_file,allow_pickle=True)

In [30]:
sentences[0].count(' ')+1==len(align_dict['words']) # the aligned words should be the same as the original prompts

True

### Method 1: extract detailed Gentle infor 

In [3]:
# parse them in a loop
phone_list_sentences=[]
failed_sentences=folder+'failed_sentences.txt'
if os.path.exists(failed_sentences):
  os.remove(failed_sentences)
sentences_file=meta_dir+'sentences/'+'sentences.npy'
sentences=np.load(sentences_file,allow_pickle=True)
for i in range(100): # 100 sentences
    json_file=folder+'json/'+str(i)+'.json'
    with open(json_file) as f:
        align_sentence=json.load(f)
    phone_list_sentence=[]
    for word in align_sentence['words']:
        if word['case']=='success':
            phone_list_word=[]
            start=word['start']
            end=word['end']
            for j, phone in enumerate(word['phones']):
                tmp={}
                tmp['sentence_index']=i
                tmp['word']=word['word']
                tmp['status']='success'
                tmp['duration']=phone['duration']
                tmp['phone']=phone['phone']
                if j==0:
                    tmp['start']=start
                    start2=start+phone['duration']
                else:
                    tmp['start']=start2
                    start2=start2+phone['duration']
                phone_list_word.append(tmp)
            phone_list_sentence.append(phone_list_word)
        else:
            with open(failed_sentences,'a') as f:
                f.write("%s\n" %i)
            tmp={}
            tmp['sentence_index']=i
            tmp['status']=word['case']
            tmp['word']=word['word']
            
    phone_list_sentences.append(phone_list_sentence)

In [4]:
phone_list_sentences[0] # phone list in a sentence

[[{'sentence_index': 0,
   'word': 'A',
   'status': 'success',
   'duration': 0.11,
   'phone': 'ah_S',
   'start': 0.8}],
 [{'sentence_index': 0,
   'word': 'cheap',
   'status': 'success',
   'duration': 0.2,
   'phone': 'ch_B',
   'start': 0.91},
  {'sentence_index': 0,
   'word': 'cheap',
   'status': 'success',
   'duration': 0.13,
   'phone': 'iy_I',
   'start': 1.11},
  {'sentence_index': 0,
   'word': 'cheap',
   'status': 'success',
   'duration': 0.19,
   'phone': 'p_E',
   'start': 1.2400000000000002}],
 [{'sentence_index': 0,
   'word': 'tattoo',
   'status': 'success',
   'duration': 0.13,
   'phone': 't_B',
   'start': 1.51},
  {'sentence_index': 0,
   'word': 'tattoo',
   'status': 'success',
   'duration': 0.1,
   'phone': 'ae_I',
   'start': 1.6400000000000001},
  {'sentence_index': 0,
   'word': 'tattoo',
   'status': 'success',
   'duration': 0.16,
   'phone': 't_I',
   'start': 1.7400000000000002},
  {'sentence_index': 0,
   'word': 'tattoo',
   'status': 'success'

In [5]:
filename=folder+'/phone_list_all_sentences.npy'
np.save(filename, np.array(phone_list_sentences, dtype=object), allow_pickle=True)

In [6]:
# load
#filename=folder+'alignment/phone_list_all_sentences.npy'
#a=np.load(filename,allow_pickle=True)

### Method 2: simplify the phone_list_all_sentences file

In [3]:
failed_sentences=folder+'failed_sentences.txt'
if os.path.exists(failed_sentences):
  os.remove(failed_sentences)
phone_list_sentences=[]
oovs=[]
nfia=[]
for i in range(100): # 100 sentences
    json_file=folder+'json/'+str(i)+'.json'
    with open(json_file) as f:
        align_sentence=json.load(f)
    phone_list_sentence=[]
    for word in align_sentence['words']:
        if word['case']=='success': # including those oov situations
            phone_list_word=[]
            start=word['start']
            end=word['end']
            for j, phone in enumerate(word['phones']):
                tmp={}
                key=phone['phone'][:-2] # get ride of last two, eg '_S','_B', '_E','_I'
                #tmp['sentence_index']=i
                #tmp['word']=word['word']
                #tmp['status']='success'
                #tmp['duration']=phone['duration']
                #tmp['phone']=phone['phone']
                if key=='oov':
                    info=[start,0]
                    oovs.append(i)
                else:
                    if j==0:
                        start_p=start
                        start2=start+phone['duration']
                    else:
                        start_p=start2
                        start2=start2+phone['duration']
                    
                    info=[start_p,phone['duration']]
                
                tmp[key]=info
                phone_list_sentence.append(tmp)

        elif word['case']=='not-found-in-audio':
            nfia.append(i)
            try:
                start=word['start'] # manually add start time to nfia if it is right after a regular syllable: this start time will be used to calculate duration of that syllable.
            except:
                start=0
            for j, phone in enumerate(word['phones']):
                tmp={}
                info=[start,0]
                key=phone['phone'][:-2]
                tmp[key]=info
                phone_list_sentence.append(tmp)
        else:
            with open(failed_sentences,'a') as f:
                f.write("%s\n" %i)
            tmp={}
            tmp['sentence_index']=i
            tmp['status']=word['case']
            tmp['word']=word['word']
            
    phone_list_sentences.append(phone_list_sentence)

print('There are '+str(len(oovs))+' oov syllables; '+str(len(nfia))+' not-found-in-audio words.')  # Surprise that There are 145 oovs.

There are 145 oov syllables; 5 not-found-in-audio words.


In [5]:
phone_list_sentences[7] # one example oov (setence begining);

[{'ah': [0.0, 0.01]},
 {'t': [1.05, 0.08]},
 {'iy': [1.1300000000000001, 0.36]},
 {'ih': [1.84, 0.11]},
 {'z': [1.9500000000000002, 0.09]},
 {'b': [2.11, 0.08]},
 {'r': [2.19, 0.09]},
 {'uw': [2.28, 0.2]},
 {'d': [2.48, 0.15]},
 {'w': [3.11, 0.01]},
 {'ih': [3.1199999999999997, 0.01]},
 {'th': [3.1299999999999994, 0.01]},
 {'l': [3.15, 0.09]},
 {'iy': [3.2399999999999998, 0.18]},
 {'v': [3.42, 0.08]},
 {'z': [3.5, 0.22]},
 {'ao': [4.24, 0.15]},
 {'r': [4.390000000000001, 0.1]},
 {'not-found-in-aud': [4.49, 0]},
 {'not-found-in-aud': [4.49, 0]},
 {'not-found-in-aud': [4.49, 0]},
 {'not-found-in-aud': [4.49, 0]}]

In [6]:
nfia # [7*3, 36*2, 61*8, 61, 61]=3+2+8=13 phones


[7, 36, 61, 61, 61]

In [15]:
filename=folder+'/phone_list_all_sentences2.npy'
np.save(filename, np.array(phone_list_sentences, dtype=object), allow_pickle=True)

## add on/off beat marks

In [16]:
import pandas
import copy

In [17]:
sid=2
folder=meta_dir+'SEEG'+str(sid)+'/forced_alignment/' #alignment/'
filename=folder+'/phone_list_all_sentences2.npy'
phone_list_gentle=np.load(filename,allow_pickle=True)

In [18]:
# read the excel file
filename=meta_dir+'sentences/sentences_v3.xlsx'
tmp=pandas.read_excel(filename)
sentences_para=tmp.to_numpy()[0:-1,1:9]

In [19]:
# find the nan entries
nans=[i for i, sentence in enumerate(sentences_para) for syllable in sentence if isinstance(syllable,float) ]
print(nans) # [37, 82]

[37, 82]


In [20]:
counts=[len(syllable.split('|')[1:-1]) for sentence in sentences_para for syllable in sentence if not isinstance(syllable,float)]
sum(counts) # 2181 phones

2181

In [21]:
tmp=[]
for i, sentence in enumerate(phone_list_gentle):
    for phone in sentence:
        if list(phone.keys())[0]=='oov':
            tmp.append(i)
print(len(tmp)) # 145 oov

145


In [22]:
tmp # files with oov: [1, 12, 12, 29, 38, 51, 52, 52, 54, 58, 58, 62, 62, 63, 72, 72, 75, 76, 77, 79, 80, 82, 82, 84, 86, 86, 88, 89]
# Manually add the missing phonemes of the oov word to Gentle JSON file.

[1,
 1,
 1,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 29,
 29,
 29,
 29,
 38,
 38,
 38,
 51,
 51,
 51,
 51,
 51,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 54,
 54,
 54,
 54,
 54,
 58,
 58,
 58,
 58,
 58,
 58,
 58,
 58,
 58,
 58,
 58,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 62,
 63,
 63,
 63,
 63,
 63,
 63,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 72,
 75,
 75,
 75,
 75,
 75,
 75,
 76,
 76,
 76,
 76,
 76,
 76,
 76,
 77,
 77,
 77,
 77,
 77,
 79,
 79,
 79,
 79,
 80,
 80,
 80,
 80,
 80,
 80,
 82,
 82,
 82,
 82,
 82,
 82,
 82,
 82,
 82,
 82,
 84,
 84,
 84,
 84,
 84,
 84,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 86,
 88,
 88,
 88,
 88,
 88,
 89,
 89,
 89,
 89,
 89]

In [23]:
# gentle file might still contain "not-found-in-audio", check it.
# Manually add missing phonemes (belongs to the not-found-in-audio word) to Gentle file.


In [24]:
# the length between Gentle and paradigm might not the same: this might because one phoneme from one file were split in its counterpart
# Found these files: 11 15 30 38 43; Corrected;
# Modify the Gentle file to reflect the paradigm sequence.
import math
for i,phone_list_para in enumerate(sentences_para):
    tt1=0
    for syllable in phone_list_para:
        if isinstance(syllable,float):
            if math.isnan(syllable):
                continue
        else:
            for phone in syllable.split('|')[1:-1]:
                tt1+=1
    #tt1=sum([1 for syllable in phone_list_para if isinstance(syllable,float) and not math.isnan(syllable) for phone in syllable.split('|')[1:-1]])
    tt2=len(phone_list_gentle[i])
    if tt1!=tt2:
        print(i)
        

In [25]:
# read Gentle and excel, and add on/off beat info in the excel to Gentle
phone_list_sentences2=[]
for i,phone_list_para in enumerate(sentences_para):
    phones_aligned=phone_list_gentle[i] # list of dicts: ['phone':[start, duration], 'phone':[], 'phone':[], ]
    phones_aligned2=copy.deepcopy(phones_aligned)# list of dicts: ['phone':[start, duration, on(1)/off(-1) beat], 'phone':[], 'phone':[], ]
    beat=-1
    index=0
    for j,syllable in enumerate(phone_list_para):
        if not isinstance(syllable,float): # one of the syllable is nan (a blank in the excel sheet)
            for k,phone in enumerate(syllable.split('|')[1:-1]):
                phone_aligned=list(phones_aligned[index].keys())[0]
                if phone_aligned=='oov':
                    info=phones_aligned[index]['oov']
                    info.append(beat) # denote oov as 0
                    phones_aligned2[index]['oov']=info
                    index=index+1
                elif phone_aligned != phone:
                    print(str(i)+ ' key mismatch: '+phone_aligned+' is not '+phone+'.')
                    info=phones_aligned[index][phone_aligned] # time
                    info.append(beat) # mark
                    if phone_aligned =='not-found-in-aud':
                        phones_aligned2[index]['not-found-in-aud']=info
                    else:
                        phones_aligned2[index].pop(phone_aligned)
                        phones_aligned2[index][phone]=info
                    index=index+1
                else: # todo: mark the a/i/u instance;
                    info=phones_aligned[index][phone]
                    info.append(beat)
                    phones_aligned2[index][phone]=info
                    index=index+1
                
            beat=beat*(-1)
    phone_list_sentences2.append(phones_aligned2)         
        

7 key mismatch: not-found-in-aud is not b.
7 key mismatch: not-found-in-aud is not ae.
7 key mismatch: not-found-in-aud is not g.
7 key mismatch: not-found-in-aud is not z.
12 key mismatch: ah is not ae.
15 key mismatch: ah is not ae.
17 key mismatch: ah is not ih.
18 key mismatch: ih is not ah.
20 key mismatch: uw is not ih.
21 key mismatch: ih is not uw.
23 key mismatch: ah is not ae.
23 key mismatch: ah is not ae.
25 key mismatch: ih is not ah.
26 key mismatch: ey is not ah.
26 key mismatch: ah is not ae.
27 key mismatch: ah is not ae.
29 key mismatch: ah is not ae.
29 key mismatch: ah is not ey.
30 key mismatch: uw is not ih.
34 key mismatch: ah is not ae.
36 key mismatch: not-found-in-aud is not m.
36 key mismatch: not-found-in-aud is not uw.
39 key mismatch: ah is not ae.
41 key mismatch: ah is not ae.
42 key mismatch: ah is not ae.
42 key mismatch: ah is not ae.
45 key mismatch: ah is not iy.
48 key mismatch: uh is not ao.
50 key mismatch: ih is not ah.
51 key mismatch: ah is no

### below are some key errors:

7 key error: not-found-in-aud is not b.
7 key error: not-found-in-aud is not ae.
7 key error: not-found-in-aud is not g.
7 key error: not-found-in-aud is not z.
12 key error: ah is not ae.
15 key error: ah is not ae.
17 key error: ah is not ih.
18 key error: ih is not ah.
20 key error: uw is not ih.
21 key error: ih is not uw.
23 key error: ah is not ae.
23 key error: ah is not ae.
25 key error: ih is not ah.
26 key error: ey is not ah.
26 key error: ah is not ae.
27 key error: ah is not ae.
29 key error: ah is not ae.
29 key error: ah is not ey.
30 key error: uw is not ih.
34 key error: ah is not ae.
36 key error: not-found-in-aud is not m.
36 key error: not-found-in-aud is not uw.
39 key error: ah is not ae.
41 key error: ah is not ae.
42 key error: ah is not ae.
42 key error: ah is not ae.
45 key error: ah is not iy.
48 key error: uh is not ao.
50 key error: ih is not ah.
51 key error: ah is not ae.
52 key error: ah is not ae.
54 key error: ah is not ae.
54 key error: ah is not ae.
55 key error: ah is not ae.
56 key error: ih is not ah.
61 key error: ah is not ae.
61 key error: iy is not ah.
61 key error: not-found-in-aud is not m.
61 key error: not-found-in-aud is not ae.
61 key error: not-found-in-aud is not ch.
61 key error: not-found-in-aud is not f.
61 key error: not-found-in-aud is not ao.
61 key error: not-found-in-aud is not r.
61 key error: not-found-in-aud is not y.
61 key error: not-found-in-aud is not uw.
62 key error: ah is not ae.
62 key error: ah is not ey.
65 key error: ih is not ah.
66 key error: ah is not ae.
70 key error: ah is not ae.
73 key error: ah is not ae.
74 key error: ah is not ae.
75 key error: ah is not ae.
76 key error: ih is not ah.
79 key error: ey is not ah.
86 key error: ah is not iy.
87 key error: ah is not ae.
88 key error: uw is not ah.
91 key error: iy is not ih.
92 key error: ah is not ae.
92 key error: ih is not ah.
96 key error: ah is not ae.
98 key error: ah is not ae.


When differs, use the paradigm phones.

In [27]:
# check
phone_list_sentences2[7]

[{'ah': [0.0, 0.01, -1]},
 {'t': [1.05, 0.08, 1]},
 {'iy': [1.1300000000000001, 0.36, 1]},
 {'ih': [1.84, 0.11, -1]},
 {'z': [1.9500000000000002, 0.09, -1]},
 {'b': [2.11, 0.08, 1]},
 {'r': [2.19, 0.09, 1]},
 {'uw': [2.28, 0.2, 1]},
 {'d': [2.48, 0.15, 1]},
 {'w': [3.11, 0.01, -1]},
 {'ih': [3.1199999999999997, 0.01, -1]},
 {'th': [3.1299999999999994, 0.01, -1]},
 {'l': [3.15, 0.09, 1]},
 {'iy': [3.2399999999999998, 0.18, 1]},
 {'v': [3.42, 0.08, 1]},
 {'z': [3.5, 0.22, 1]},
 {'ao': [4.24, 0.15, -1]},
 {'r': [4.390000000000001, 0.1, -1]},
 {'not-found-in-aud': [4.49, 0, 1]},
 {'not-found-in-aud': [4.49, 0, 1]},
 {'not-found-in-aud': [4.49, 0, 1]},
 {'not-found-in-aud': [4.49, 0, 1]}]

In [28]:
filename=folder+'/phone_list_onoff_beat.npy'
np.save(filename, np.array(phone_list_sentences2, dtype=object), allow_pickle=True)